In [2]:
pip install fake_useragent

In [ ]:
from os import name
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import progressbar
import requests
from bs4 import BeautifulSoup
import fake_useragent
import time
import json
from google.colab import drive

name_vacancies = ['"ML-инженер" OR "Data scientist" OR "Data engineer"', "Тестировщик", "Бизнес аналитик", "DevOps инженер", "IOS разработчик", "Android разработчик", '"продуктовый аналитик" OR "product analyst"']
for item in name_vacancies:
  url = 'https://api.hh.ru/vacancies'

  params = {
            'text': item,
            'per_page': 100,
            'page': 0,
            'area': 113,
            'currency': 'RUR',
            'date_from': '2023-06-01',
            'date_to': '2023-12-31',
            'only_with_salary': 'True'
            }

  resp = requests.get(url = url, params = params)
  answer = resp.text
  json_text = json.loads(answer)
  items = []
  count_pages = json_text['pages']
  for i in range(count_pages):
    params['page'] = i
    resp = requests.get(url=url, params=params)
    print('page: ', i, ', status_code: ', resp.status_code)
    json_text = json.loads(resp.text)
    items.extend(json_text['items'])
  # df = pd.DataFrame.from_dict(items, orient='columns')
  df = pd.json_normalize(items)
  prof_role_dict = pd.json_normalize(df['professional_roles'])
  prof_role_dict.columns = ['name']
  prof_role_name = pd.json_normalize(prof_role_dict['name'])
  prof_role_name.rename(columns={'id': 'professional_roles.id', 'name': 'professional_roles.name'}, inplace=True)
  # working_time_modes = pd.json_normalize(df['working_time_modes'])
  # working_time_modes.columns = ['working_time_modes']
  # working_time_modes = pd.json_normalize(working_time_modes['working_time_modes'])
  # working_time_modes.rename(columns={'id': 'working_time_modes.id', 'name': 'working_time_modes.name'}, inplace=True)
  df['working_time_modes.id'] = 'NaN'
  # df['working_time_modes.name'] = 'NaN'
  df['salary'] = 'NaN'
  # working_time_intervals = pd.json_normalize(df['working_time_intervals'])
  # working_time_intervals.columns = ['working_time_intervals']
  # working_time_intervals = pd.json_normalize(working_time_intervals['working_time_intervals'])
  # working_time_intervals.rename(columns={'id': 'working_time_intervals.id', 'name': 'working_time_intervals.name'}, inplace=True)
  df_combined = pd.concat([df, prof_role_name], axis=1)
  df_selected = df_combined[['name', 'id', 'premium', 'alternate_url', 'created_at','employer.name', 'area.name','professional_roles.name', 'employment.name', 'experience.name', 'salary', 'salary.from','salary.to', 'salary.currency', 'salary.gross', 'schedule.name', 'snippet.requirement', 'snippet.responsibility', 'type.name']]

  bar = progressbar.ProgressBar(max_value=len(df_selected['id']))

  def get_links(id):
    try:
      usag = fake_useragent.UserAgent()
      data = requests.get(url = f"https://hh.ru/vacancy/{id}",
                      # "https://hh.ru/vacancy/88796932",
                      headers = {"user-agent":usag.random})
      resp = data.content
      # resp.raise_for_status()
      b = BeautifulSoup(resp, 'lxml')
      vacancy_description = b.find('div', attrs={"data-qa":"vacancy-description"}).get_text()
    except Exception as e:
      print(f"Ошибка при запросе: {e}")
      # continue

    else:
      return vacancy_description

  descriptions = []

  # if __name__=="__main__":
  for i, id in enumerate(list(df_selected['id'])):
    if pd.isna(id) or id == "" or id == "NA":
      continue
    else:
      time.sleep(1)
      descriptions.append([id, get_links(id)])
      res = pd.DataFrame(np.asarray(descriptions), columns = ['id', 'description.vacancy'])
      bar.update(i+1)

  merged_df = df_selected.merge(res, on='id')

drive.mount('/content/drive')
merged_df.to_csv(f'/content/drive/MyDrive/Project/{item}.csv', index=False)


page:  0 , status_code:  200


 84% (22 of 26) |####################    | Elapsed Time: 0:00:51 ETA:   0:00:09

Ошибка при запросе: 'NoneType' object has no attribute 'get_text'


100% (26 of 26) |########################| Elapsed Time: 0:01:01 ETA:  00:00:00

page:  0 , status_code:  200
page:  1 , status_code:  200
page:  2 , status_code:  200


 13% (35 of 269) |##                     | Elapsed Time: 0:01:24 ETA:   0:08:59